## Omega and Xi

To implement Graph SLAM, a matrix and a vector (omega and xi, respectively) are introduced. The matrix is square and labelled with all the robot poses (xi) and all the landmarks (Li). Every time you make an observation, for example, as you move between two poses by some distance `dx` and can relate those two positions, you can represent this as a numerical relationship in these matrices.

Below you can see a matrix representation of omega and a vector representation of xi.

<img src='images/omega_xi.png' width=20% height=20% />


### Solving for x, L

To "solve" for all these poses and landmark positions, we can use linear algebra; all the positional values are in the vector `mu` which can be calculated as a product of the inverse of omega times xi.

---


## Constraint Updates

In the below code, we construct `omega` and `xi` constraint matrices, and update these according to landmark sensor measurements and motion.

#### Sensor Measurements

When you sense a distance, `dl`, between a pose and a landmark, l, update the constraint matrices as follows:
* Add `[[1, -1], [-1, 1]]` to omega at the indices for the intersection of `xt` and `l`
* Add `-dl` and `dl` to xi at the rows for `xt` and `l`

The values 2 instead of 1 indicate the "strength" of the measurement.

You'll see three new `dl`'s as new inputs to our function `Z0, Z1, Z2`, below.

#### Motion
When your robot moves by some amount `dx` update the constraint matrices as follows:
* Add `[[1, -1], [-1, 1]]` to omega at the indices for the intersection of `xt` and `xt+1`
* Add `-dx` and `dx` to xi at the rows for `xt` and `xt+1`

In [26]:
import numpy as np
from numpy.linalg import inv

def init(omega, xi, d):
    # update omega 
    omega[0, 0] = 1
    
    # update xi
    xi[0] = d

def update(omega, xi, x1, x2, d, strength=1):
    # update omega
    omega[x1, x1] += strength
    omega[x2, x2] += strength
    omega[x1, x2] -= strength
    omega[x2, x1] -= strength
    
    # update xi
    xi[x1] -= d
    xi[x2] += d


def mu_from_positions(initial_pos, move1, move2, Z0, Z1, Z2):
    
    ## construct constraint matrices
    ## and add each position/motion constraint to them
    omega = np.zeros((4, 4))
    xi = np.zeros((4, 1))
    
    # initial position
    init(omega, xi, initial_pos)
    # observe the landmark from the initial position
    update(omega, xi, 0, 3, Z0)
    
    # move x0 -> x1
    update(omega, xi, 0, 1, move1)
    # observe the landmark from the position 1
    update(omega, xi, 1, 3, Z1)
    
    # move x1 -> x2
    update(omega, xi, 1, 2, move2)
    # observe the landmark from the position 2
    # The last sensor measurement (Z2) is extremely confident.
    # Multiple the "strength" of that measurement by a factor of 5.
    update(omega, xi, 2, 3, Z2, 5)
    
    # display final omega and xi
    print('Omega: \n', omega, '\n')
    print('Xi: \n', xi, '\n')
    
    ## calculate mu as the inverse of omega * xi
    mu = np.dot(inv(omega), xi)
    
    return mu


In [29]:
# call function and print out `mu`
mu = mu_from_positions(-3, 5, 3, 10, 5, 1)
print('Mu: \n', mu)

Omega: 
 [[ 3. -1.  0. -1.]
 [-1.  3. -1. -1.]
 [ 0. -1.  6. -5.]
 [-1. -1. -5.  7.]] 

Xi: 
 [[-18.]
 [ -3.]
 [  2.]
 [ 16.]] 

Mu: 
 [[-3.        ]
 [ 2.32142857]
 [ 6.28571429]
 [ 6.67857143]]
